<a href="https://colab.research.google.com/github/victorgrrtj/bi_lui_work/blob/main/Trabalho_LUI_BI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho BI e LUI

## Instalação e Importação de Bibliotecas

In [ ]:
# Instalação do pacote Wordcloud
!pip install wordcloud -q

In [ ]:
# Importação das bibliotecas
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import re
from IPython.display import HTML
import math
import time
import random
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "Accept-Language": "en-US,en;q=0.5", "Accept-Encoding": "gzip, deflate", "DNT": "1", "Connection": "close", "Upgrade-Insecure-Requests": "1"}

## Detecção e Extração de Dados

In [ ]:
#Criação de Request para extração do código HTML
response = requests.get('https://www.dfimoveis.com.br/aluguel/df/todos/imoveis')
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
#Extração de Quantidade de Imóveis em todas as páginas
qtd_imoveis = soup.find('h1')
qtd_imoveis = int(qtd_imoveis.text.strip().split(' ')[0].replace('.', ''))

In [ ]:
#Painel de Informações a serem extraídas
painel = soup.find('div', attrs={'class': 'result-search__content'})

In [ ]:
#Extração do item que contém os dados de cada imóvel a serem inseridos no dataframe
imoveis = painel.find_all('li', attrs={'class': 'property-list__item'})

In [ ]:
# Imprime a quantidade de imóveis em 1 página
print(len(imoveis))

In [ ]:
# Quantidade de páginas para todos imóveis cadastrados
n_pag = qtd_imoveis / len(imoveis)
n_pag = int(math.ceil(n_pag))
print("São necessárias {} páginas para conter {} imóveis.".format(n_pag, qtd_imoveis))

In [ ]:
# Lista de URL's a serem submetidas ao Web Scrapping
url_list = []
for i in range(1, n_pag + 1):
    url_list.append('https://www.dfimoveis.com.br/aluguel/df/todos/imoveis?pagina='+str(i))
print(url_list)

In [ ]:
#Extrai link da página do imóvel
extracted_data = []
for imovel in imoveis:
  result = {}
  hyperlink_format = '<a href="{link}">{text}</a>'
  link = "https://www.dfimoveis.com.br"+imovel.find('a', attrs={'class': 'gtm-imovel texto-uma-linha'}).get('href')
  link = hyperlink_format.format(link = link, text="LINK")
  result['link'] = link
  extracted_data.append(result)

In [ ]:
#Extrai título do imóvel
extracted_data = []
for imovel in imoveis:
  result = {}
  result['titulo'] = imovel.find('a', attrs={'class': 'gtm-imovel texto-uma-linha'}).text.strip()
  extracted_data.append(result)

In [ ]:
#Extrai preço mensal de aluguel ou total de venda
extracted_data = []
for imovel in imoveis:
  result = {}
  precos = imovel.find('h4', attrs={'class': 'property__subtitle hide-mobile'})
  result['preco_mensal'] = int(re.findall('[0-9]+', precos.find_all('span', attrs={'class': 'price'})[0].text.replace('.', ''))[0])
  extracted_data.append(result)

In [ ]:
#Extrai preço por m2
extracted_data = []
for imovel in imoveis:
  result = {}
  precos = imovel.find('h4', attrs={'class': 'property__subtitle hide-mobile'})
  try:
    result['preco_m2'] = int(re.findall('[0-9]+', precos.find_all('span', attrs={'class': 'price'})[1].text.replace('.', ''))[0])
  except IndexError as error:
    result['preco_m2'] = None
  extracted_data.append(result)

In [ ]:
#Extrai objetivo (aluguel, venda, etc), tipo de imovel (casa, apto, sala), quantos m2 e tipo/qtde de cômodos
extracted_data = []
for imovel in imoveis:
  result = {}
  caracts = imovel.find_all('ul', attrs={'class': 'property__options'})
  caracts = caracts[1].find_all('li')
  caract_1 = caracts[0].text.split(' ')
  result['obj'] = caract_1[0]
  result['tipo'] = caract_1[1]
  result['m2'] = int(re.findall('[0-9]+', caracts[0].text)[0])
  if len(caracts) > 1:
    for caract in caracts[1:]:
      split = caract.text.split(' ')
      qtd = int(split[0])
      comodo = split[1].lower()
      if comodo in ['quarto', 'suíte', 'vaga']:
        comodo = comodo+'s'
      result[comodo] = qtd
  extracted_data.append(result)

In [ ]:
# Extrai conteúdo (descrição do imóvel)
extracted_data = []
for imovel in imoveis:
  result = {}
  result['descricao'] = imovel.find('meta', attrs={'itemprop': 'description'}).get('content').replace('\r', '').replace('\n', '')
  extracted_data.append(result)

## Sumarização de todas as extrações.

In [ ]:
""" Contém looping para todas as páginas.
Retorna um dicionário com as informações de cada imóvel.
Devido à quantidade de páginas, este processo pode demorar mais de 30 minutos."""

extracted_data = []
response = requests.get('https://www.dfimoveis.com.br/aluguel/df/todos/imoveis')
soup = BeautifulSoup(response.text, 'html.parser')
qtd_imoveis = soup.find('h1', attrs={'itemprop': 'name'})
qtd_imoveis = int(qtd_imoveis.text.strip().split(' ')[0].replace('.', ''))
painel = soup.find('div', attrs={'class': 'result-search__content'})
imoveis = painel.find_all('li', attrs={'class': 'property-list__item'})

n_pag = math.ceil(qtd_imoveis / len(imoveis))
print("São necessárias {} páginas para conter {} imóveis.".format(n_pag, qtd_imoveis))

url_list = []
for i in range(1, n_pag + 1):
  url_list.append('https://www.dfimoveis.com.br/aluguel/df/todos/imoveis?pagina='+str(i))

for index, url in enumerate(url_list):
  response = requests.get(url, headers=headers)
  print("Codigo de resposta da pagina: ", response.status_code)
  soup = BeautifulSoup(response.text, 'html.parser')
  painel = soup.find('div', attrs={'class': 'result-search__content'})
  imoveis = painel.find_all('li', attrs={'class': 'property-list__item'})
  time.sleep(random.randint(2,6))

  for imovel in imoveis:
    result = {}

    hyperlink_format = '<a href="{link}">{text}</a>'
    link = "https://www.dfimoveis.com.br"+imovel.find('a', attrs={'class': 'gtm-imovel texto-uma-linha'}).get('href')
    link = hyperlink_format.format(link = link, text="LINK")
    try:
      result['link'] = link
    except:
      result['link'] = None

    try:
      result['titulo'] = imovel.find('a', attrs={'class': 'gtm-imovel texto-uma-linha'}).text.strip()
    except:
      result['titulo'] = None

    precos = imovel.find('h4', attrs={'class': 'property__subtitle hide-mobile'})
    try:
      result['preco_mensal'] = int(re.findall('[0-9]+', precos.find_all('span', attrs={'class': 'price'})[0].text.replace('.', ''))[0])
    except:
      result['preco_mensal'] = None

    try:
      result['preco_m2'] = int(re.findall('[0-9]+', precos.find_all('span', attrs={'class': 'price'})[1].text.replace('.', ''))[0])
    except IndexError as error:
      result['preco_m2'] = None

    caracts = imovel.find_all('ul', attrs={'class': 'property__options'})
    caracts = caracts[1].find_all('li')
    caract_1 = caracts[0].text.split(' ')
    try:
      result['obj'] = caract_1[0]
    except:
      result['obj'] = None
    try:
      result['tipo'] = caract_1[1]
    except:
      result['tipo'] = None
    try:
      result['m2'] = int(re.findall('[0-9]+', caracts[0].text)[0])
    except IndexError as error:
      result['m2'] = None
    if len(caracts) > 1:
      for caract in caracts[1:]:
        split = caract.text.split(' ')
        qtd = int(split[0])
        comodo = split[1].lower()
        if comodo in ['quarto', 'suíte', 'vaga']:
          comodo = comodo+'s'
        result[comodo] = qtd
    result['descricao'] = imovel.find('meta', attrs={'itemprop': 'description'}).get('content').replace('\r', '').replace('\n', '')

    extracted_data.append(result)
  print('Finished page ' + str(index + 1))
print('Finished all pages')

In [ ]:
# Salva o resultado em um arquivo Json.
with open('data.json', 'w') as outfile:
    json.dump(extracted_data, outfile, ensure_ascii=False)

In [ ]:
# Transforma o dicionário com os dados extraídos em um DataFrame
df = pd.json_normalize(extracted_data)
#para formatar link no df
HTML(df.to_html(escape=False))

## Wordcloud das descrições dos imóveis.

In [ ]:
# Criação de Dataframe contendo apenas as descrições.
summary = df.dropna(subset=['summary'], axis=0)['summary']

In [ ]:
# Exemplos de descrições dos imóveis.
display(summary.iloc[100])
display(summary.iloc[120])
display(summary.iloc[133])

In [ ]:
# Concatenar as descrições.
all_summary = " ".join(s for s in summary)

# Quantidade de palavras
print("Quantidade de Palavras: {}".format(len(all_summary)))

In [ ]:
# Lista de stopword
stopwords = set(STOPWORDS)
stopwords.update(["da", "meu", "em", "você", "de", "ao", "os", "sua"])

# gerar uma wordcloud
wordcloud = WordCloud(stopwords=stopwords,
                      background_color="black",
                      width=1600, height=800).generate(all_summary)

# mostrar a imagem final
fig, ax = plt.subplots(figsize=(20,12))
ax.imshow(wordcloud, interpolation='bilinear')
ax.set_axis_off()

plt.imshow(wordcloud);
wordcloud.to_file("df_descricoes.png")

## Wordcloud dos títulos dos imóveis.

In [ ]:
# Criação de Dataframe contendo apenas os títulos.
titles = df.dropna(subset=['titulo'], axis=0)['titulo']

In [ ]:
# Exemplos de títulos de imóveis.
display(titles.iloc[100])
display(titles.iloc[120])
display(titles.iloc[133])

In [ ]:
# Concatenar os títulos..
all_titles = " ".join(s for s in titles)

# Quantidade de palavras
print("Quantidade de Palavras: {}".format(len(all_titles)))

In [ ]:
# Lista de stopword
stopwords = set(STOPWORDS)
stopwords.update(["da", "meu", "em", "você", "de", "ao", "os", "sua", "bloco", "rua", "quadra"])

# gerar uma wordcloud
wordcloud = WordCloud(stopwords=stopwords,
                      background_color="black",
                      width=1600, height=800).generate(all_titles)

# mostrar a imagem final
fig, ax = plt.subplots(figsize=(20,12))
ax.imshow(wordcloud, interpolation='bilinear')
ax.set_axis_off()

plt.imshow(wordcloud);
wordcloud.to_file("df_titulos.png")

## Wordcloud dos tipos dos imóveis.

In [ ]:
# Criação de Dataframe contendo apenas os tipos.
types = df.dropna(subset=['tipo'], axis=0)['tipo']

In [ ]:
# Exemplos de tipos de imóveis.
display(types.iloc[100])
display(types.iloc[120])
display(types.iloc[133])

In [ ]:
# Concatenar os tipos.
all_types = " ".join(s for s in types)

# Quantidade de palavras
print("Quantidade de Palavras: {}".format(len(all_types)))

In [ ]:
# Lista de stopword
stopwords = set(STOPWORDS)
stopwords.update(["da", "meu", "em", "você", "de", "ao", "os", "sua", "bloco", "rua", "quadra"])

# gerar uma wordcloud
wordcloud = WordCloud(stopwords=stopwords,
                      background_color="black",
                      width=1600, height=800).generate(all_types)

# mostrar a imagem final
fig, ax = plt.subplots(figsize=(12,8))
ax.imshow(wordcloud, interpolation='bilinear')
ax.set_axis_off()

plt.imshow(wordcloud);
wordcloud.to_file("df_tipos.png")